In [44]:
import shapely
import geopandas as gpd
import matplotlib.pyplot as plt
import shapefile as sh

In [41]:
test = sh.Reader("D:/GIS_Data/Weltwald/EU_BOT_point.shp")
test.fields


[('DeletionFlag', 'C', 1, 0),
 ['ID_PROC', 'F', 19, 11],
 ['ID_tree', 'F', 19, 11],
 ['CONIF', 'F', 19, 11],
 ['SEL', 'F', 19, 11],
 ['X_ACHSE', 'C', 10, 0],
 ['Y_ACHSE', 'C', 10, 0],
 ['X_QUADRAT', 'C', 10, 0],
 ['Y_QUADRAT', 'C', 10, 0],
 ['PARZELLE', 'C', 10, 0],
 ['PARZELLE_1', 'C', 10, 0],
 ['ID_BA_1', 'C', 17, 0],
 ['BA_1_HERKU', 'C', 101, 0],
 ['BA_1_PFLAN', 'C', 10, 0],
 ['UMBAU', 'C', 10, 0],
 ['ID_BA_2', 'C', 10, 0],
 ['BA_2_HERKU', 'C', 10, 0],
 ['BA_2_PFLAN', 'C', 10, 0],
 ['BA_1_DOKU', 'C', 10, 0],
 ['BA_2_DOKU', 'C', 10, 0],
 ['BA_1_TAXON', 'C', 60, 0],
 ['BA_2_TAXON', 'C', 13, 0],
 ['BA_1_ART', 'C', 51, 0],
 ['BA_2_SYNON', 'C', 10, 0],
 ['BA_1_SYNON', 'C', 15, 0],
 ['BA_1_SYSTE', 'C', 10, 0],
 ['BA_2_SYSTE', 'C', 10, 0],
 ['BA_1_BOT_M', 'C', 10, 0],
 ['BA_2_BOT_M', 'C', 10, 0],
 ['BA_1_GEOGR', 'C', 10, 0],
 ['BA_2_GEOGR', 'C', 10, 0],
 ['BA_1_OEKOL', 'C', 10, 0],
 ['BA_2_OEKOL', 'C', 10, 0],
 ['BA_1_LINK_', 'C', 62, 0],
 ['BA_2_LINK_', 'C', 10, 0],
 ['BA_1_LINK1', 'C', 1

In [46]:
'''
生成矩形框
'''
test = sh.Reader("D:/GIS_Data/Weltwald/EU_BOT_point.shp")

out = 'test.shp'

w = sh.Writer(out)
w.field('tree species')

width = 5
length = 5
for item in test.shapes():
    x = item.points[0][0]
    y = item.points[0][1]
    x_left_up,y_left_up = x-width,y-length
    x_left_down,y_left_down = x-width,y+length
    x_right_up,y_right_up = x+width,y-length
    x_right_down,y_right_down = x+width,y+length
    w.poly([
        [[x_left_up,y_left_up],[x_left_down,y_left_down],[x_right_down,y_right_down],[x_right_up,y_right_up]]
    ])
    w.record(str(item._Shape__oid))
w.close()


In [47]:
'''
矩形框裁剪遥感影像
'''

import rasterio as rio
from rasterio.mask import mask
import numpy as np
from PIL import Image
import os


In [48]:
shpdata = r'test.shp'
shpdata = gpd.read_file(shpdata)
# polygon = shpdata.geometry[0]

for i in range(0,shpdata.shape[0]):
    polygon = shpdata.geometry[i]
    with rio.open(r'D:\UAV_Data\Weltwald\Europa_BOT\RGB\20220614_MS_RGB_EU_BOT_85_85_V3_transparent_mosaic_group1.tif') as rasterdata:
        out_crs = rasterdata.crs
        feature = [polygon.__geo_interface__]
        out_image,out_transform = mask(rasterdata,feature,all_touched=True,crop=True,nodata=255)
        out_image_PIL = Image.fromarray(np.transpose(out_image,(1,2,0))).convert('RGB')
        save_path = os.path.join(r'D:\GIS_Data\Weltwald\test','raster_'+str(i)+'.tif')
        out_image_PIL.save(save_path,quality=95,optimize=True,progressive=True)